In [1]:
from nustar_pysolar import planning, io
import astropy.units as u
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2018_267.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [4]:
tstart = '2018-05-29T15:37:00'
tend = '2018-05-29T23:10:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

Error in function: sunlight_periods
No dates found in range. Pick a different occultation file.


# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [ ]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [ ]:
offset = [0., 0.]*u.arcsec

# Loop over each orbit and correct the pointing for the same heliocentric pointing position.

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

In [ ]:
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, parallax_correction=True)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

# This is where you actually make the Mosaic

In [ ]:
# Just use the first orbit...or choose one. This may download a ton of deltat.preds, which is a known 
# bug to be fixed.

orbit = orbits[0]
make_mosaic(orbits[0], write_output=True, make_regions=True)